In [1]:
from io import BytesIO
import numpy as np
import h5py
import png
import google.cloud.storage
from google.cloud import storage
import io
import matplotlib.pyplot as plt
import cv2
import os
import time
from IPython.display import clear_output
from PIL import Image
import tensorflow as tf
import pandas as pd
import octane_research_api.octane.api.core as octane
import cv2

In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [10]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  4


In [4]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [5]:
BUCKET_NAME = 'research-datasets-3e92f8dd-ds-25' 

In [6]:
def download_oct_images_to_file(bucket_name, prefix, delimiter=None):

    storage_client = storage.Client()

    blobs = storage_client.list_blobs(
        bucket_name, prefix=prefix, delimiter=delimiter
    )
    
    print("Blobs:")
    
    file_name = 1
    count = 0
    
    for blob in blobs:
        if count <1001:
            try:
                bytesIO = io.BytesIO(blob.download_as_string())
                h5 = h5py.File(bytesIO, 'r')
                h5_oct = h5['oct']
                png.from_array(h5_oct[64], 'L').save("DS_25/" + "AMD_image_" + str(file_name) + ".png")
                count += 1
                file_name += 1
            except:
                pass

In [7]:
download_oct_images_to_file(BUCKET_NAME, 'OutputImages')

Blobs:


In [8]:
path = os.path.join("DS_25")
IMG_SIZE = 512

final_file_name = 1

for img in os.listdir(path):
    img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    png.from_array(new_array, 'L').save("AMD_stylegan/" + "AMD_image_" + str(final_file_name) + ".png")
    final_file_name += 1

In [8]:
import argparse
import numpy as np
import PIL.Image
import re
import sys
import pretrained_networks

ModuleNotFoundError: No module named 'pretrained_networks'

In [19]:
def hackery(network_pkl, seeds, truncation_psi):
    print('Loading networks from "%s"...' % network_pkl)
    _G, _D, Gs = pretrained_networks.load_networks(network_pkl)
    noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]

    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_kwargs.randomize_noise = False
    if truncation_psi is not None:
        Gs_kwargs.truncation_psi = truncation_psi

    for seed_idx, seed in enumerate(seeds):
        print('Generating image for seed %d (%d/%d) ...' % (seed, seed_idx, len(seeds)))
        rnd = np.random.RandomState(seed)
        z = rnd.randn(1, *Gs.input_shape[1:]) # [minibatch, component]
        tflib.set_vars({var: rnd.randn(*var.shape.as_list()) for var in noise_vars}) # [height, width]
        images = Gs.run(z, None, **Gs_kwargs) # [minibatch, height, width, channel]
        PIL.Image.fromarray(images[0], 'RGB').save(dnnlib.make_run_dir_path('seed%04d.png' % seed))

In [20]:
hackery('stylegan2/results/00005-stylegan2-custom_dataset-1gpu-config-f/network-snapshot-000505.pkl', 2, 0.7)

Loading networks from "stylegan2/results/00005-stylegan2-custom_dataset-1gpu-config-f/network-snapshot-000505.pkl"...


NameError: name 'pretrained_networks' is not defined